In [6]:
import pandas as pd
import os
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn import tree
import datetime
import re


from bs4 import BeautifulSoup

import requests

from urllib.request import urlopen


In [1]:
!pwd

/Users/alecnigh/Documents/DataScienceGuidedCapstone-master/Final_Project/Capstone_project


In [2]:
%history -g

 2/1: runfile('/Users/alecnigh/hello.py', wdir='/Users/alecnigh')
 2/2: hello()
 2/3: dir()
 2/4: help(hello)
 2/5: runfile('/Users/alecnigh/hello.py', wdir='/Users/alecnigh')
 2/6: runfile('/Users/alecnigh/hello.py', wdir='/Users/alecnigh')
 2/7: runfile('/Users/alecnigh/hello.py', wdir='/Users/alecnigh')
 2/8: runfile('/Users/alecnigh/hello.py', wdir='/Users/alecnigh')
 2/9: runfile('/Users/alecnigh/hello.py', wdir='/Users/alecnigh')
2/10: runfile('/Users/alecnigh/hello.py', wdir='/Users/alecnigh')
2/11: runfile('/Users/alecnigh/hello.py', wdir='/Users/alecnigh')
2/12: runfile('/Users/alecnigh/hello.py', wdir='/Users/alecnigh')
2/13: Hello()
2/14: %reset
2/15: %reset
2/16: dir()
2/17: reset%
2/18: %reset
2/19: %reset -f
 4/1: runfile('/Users/alecnigh/hello.py', wdir='/Users/alecnigh')
 4/2: dir()
 4/3: %reset
 4/4: dir()
16/1: runfile('/Users/alecnigh/Rock paper Scissors', wdir='/Users/alecnigh')
16/2: runfile('/Users/alecnigh/Rock paper Scissors', wdir='/Users/alecnigh')
16/3: runfi

In [5]:
%history -g -f Pre-processing

File 'Pre-processing' exists. Overwrite? yes
Overwriting file.


In [7]:
df = pd.read_csv('Master_NFL_df.csv')
df.head()

,Unnamed: 0,Tm_Away,W_Away,L_Away,W-L%_Away,PF_Away,PA_Away,PD_Away,MoV_Away,SoS_Away,...,Run_Yds/G_Home,Yds/G_Defense_Home,Yds/G_Pass_D_Home,Yds/G_Run_D_Home,Yds/G_Offense_Away,Pass_Yds/G_Away,Run_Yds/G_Away,Yds/G_Defense_Away,Yds/G_Pass_D_Away,Yds/G_Run_D_Away
0,0,New England Patriots,12,4,0.75,385,237,148,9.3,1.0,...,108.6250,331.6250,201.3750,130.2500,335.5625,212.5,123.0625,294.375,200.1875,94.1875
1,1,New England Patriots,12,4,0.75,385,237,148,9.3,1.0,...,97.0000,329.5625,188.6875,140.8750,335.5625,212.5,123.0625,294.375,200.1875,94.1875
2,2,New England Patriots,12,4,0.75,385,237,148,9.3,1.0,...,104.5625,289.0625,187.9375,101.1250,335.5625,212.5,123.0625,294.375,200.1875,94.1875
3,3,New England Patriots,12,4,0.75,385,237,148,9.3,1.0,...,101.8125,355.0625,238.6250,116.4375,335.5625,212.5,123.0625,294.375,200.1875,94.1875
4,4,New England Patriots,12,4,0.75,385,237,148,9.3,1.0,...,110.1250,332.2500,159.2500,173.0000,335.5625,212.5,123.0625,294.375,200.1875,94.1875


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3989 entries, 0 to 3988
Columns: 186 entries, Unnamed: 0 to Yds/G_Run_D_Away
dtypes: float64(113), int64(62), object(11)
memory usage: 5.7+ MB


# Creating Dummy Variables

In [9]:
#Checking to see if all the data was actually dropped

filteredColumns = df.dtypes[df.dtypes == np.object]
# list of columns whose data type is object i.e. string
listOfColumnNames = list(filteredColumns.index)
print(listOfColumnNames)

['Tm_Away', 'Tm_Home', 'Date', 'Home Team', 'Away Team', 'Winner', 'Week', 'Day', 'Time', 'Winner/tie', 'Loser/tie']


<ipython-input-9-29dcc7e13d40>:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  filteredColumns = df.dtypes[df.dtypes == np.object]


In [10]:
#Editiing Week so it can still be included

df.loc[df['Week'] == 'WildCard', ['Week']] = 18
df.loc[df['Week'] == 'Division', ['Week']] = 19
df.loc[df['Week'] == 'ConfChamp', ['Week']] = 20
df.loc[df['Week'] == 'SuperBowl', ['Week']] = 21

df['Week'] = df['Week'].astype(int)

In [11]:
df['Away Winner'] = (df['Away Score'] > df['Home Score']) == 1

df['Away Winner'] = df["Away Winner"].astype(int)

In [17]:
df.drop(['Tm_Away', 'Tm_Home', 'Winner', 'Winner/tie', 'Loser/tie', 'Time'], axis = 1, inplace=True)

KeyError: "['Tm_Away' 'Tm_Home' 'Winner' 'Winner/tie' 'Loser/tie' 'Time'] not found in axis"

In [20]:
#Checking to see if all the data was actually dropped

filteredColumns = df.dtypes[df.dtypes == np.object]
# list of columns whose data type is object i.e. string
listOfColumnNames = list(filteredColumns.index)
print(listOfColumnNames)

['Home Team', 'Away Team']


<ipython-input-20-29dcc7e13d40>:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  filteredColumns = df.dtypes[df.dtypes == np.object]


In [15]:

#Time to make Date a datetime object

df['Date'] = pd.to_datetime(df['Date'])

In [19]:
df.drop('Day', axis = 1, inplace=True)

In [21]:
# Making Dummy Varibales out of our Home and Away Teams

dfo=df.select_dtypes(include=['object']) # select object type columns
df = pd.concat([df.drop(dfo, axis=1), pd.get_dummies(dfo)], axis=1)

In [22]:
#checking to make sure that our data works

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3989 entries, 0 to 3988
Columns: 250 entries, Unnamed: 0 to Away Team_Washington Redskins
dtypes: datetime64[ns](1), float64(113), int64(64), uint8(72)
memory usage: 5.7 MB


In [25]:
from sklearn.model_selection import TimeSeriesSplit

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6])
tscv = TimeSeriesSplit()
print(tscv)

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None)
TRAIN: [0] TEST: [1]
TRAIN: [0 1] TEST: [2]
TRAIN: [0 1 2] TEST: [3]
TRAIN: [0 1 2 3] TEST: [4]
TRAIN: [0 1 2 3 4] TEST: [5]


In [29]:
new = df[['Home Line Open', 'Predicted_Line', 'P_1stD%_Home', 'R_1stD%_Home','Pass_1st_D_Away','Pass_1st_D_Home', 'Away Winner', 'Away Cover']]

In [30]:
X = new[['Home Line Open', 'Predicted_Line', 'P_1stD%_Home', 'R_1stD%_Home','Pass_1st_D_Away','Pass_1st_D_Home', 'Away Winner']]
Y = new['Away Cover']

tscv = TimeSeriesSplit(n_splits=14, test_size=12)
for train_index, test_index in tscv.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[list(train_index)], X.iloc[list(test_index)]
    y_train, y_test = Y.iloc[list(train_index)], Y.iloc[list(test_index)]

    
cat_mod = tree.DecisionTreeClassifier()
cat_mod.fit(X_train, y_train)
list(zip(new.columns, cat_mod.feature_importances_))

TRAIN: [   0    1    2 ... 3818 3819 3820] TEST: [3821 3822 3823 3824 3825 3826 3827 3828 3829 3830 3831 3832]
TRAIN: [   0    1    2 ... 3830 3831 3832] TEST: [3833 3834 3835 3836 3837 3838 3839 3840 3841 3842 3843 3844]
TRAIN: [   0    1    2 ... 3842 3843 3844] TEST: [3845 3846 3847 3848 3849 3850 3851 3852 3853 3854 3855 3856]
TRAIN: [   0    1    2 ... 3854 3855 3856] TEST: [3857 3858 3859 3860 3861 3862 3863 3864 3865 3866 3867 3868]
TRAIN: [   0    1    2 ... 3866 3867 3868] TEST: [3869 3870 3871 3872 3873 3874 3875 3876 3877 3878 3879 3880]
TRAIN: [   0    1    2 ... 3878 3879 3880] TEST: [3881 3882 3883 3884 3885 3886 3887 3888 3889 3890 3891 3892]
TRAIN: [   0    1    2 ... 3890 3891 3892] TEST: [3893 3894 3895 3896 3897 3898 3899 3900 3901 3902 3903 3904]
TRAIN: [   0    1    2 ... 3902 3903 3904] TEST: [3905 3906 3907 3908 3909 3910 3911 3912 3913 3914 3915 3916]
TRAIN: [   0    1    2 ... 3914 3915 3916] TEST: [3917 3918 3919 3920 3921 3922 3923 3924 3925 3926 3927 3928]
T

[('Home Line Open', 0.14857925762292765),
 ('Predicted_Line', 0.1106871231409034),
 ('P_1stD%_Home', 0.06842042588060601),
 ('R_1stD%_Home', 0.06319613062428282),
 ('Pass_1st_D_Away', 0.09248248877766614),
 ('Pass_1st_D_Home', 0.0948657963799261),
 ('Away Winner', 0.4217687775736879)]

Here, we are seeing that the Away Winner is able to predict who covers less than half the time. Inversely, 58% of the time, the losing team may lose, but will still cover. This is importantly when people bet as they may focus on the winning more than the pointspread when deciding to bet. 

In [31]:
get_n_splits(X=X, y=Y)

NameError: name 'get_n_splits' is not defined

In [32]:
cat_mod.predict(X_test)

array([0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1])

In [34]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, cat_mod.predict(X_test))

array([[5, 1],
       [1, 5]])

# How do we split by individual season, if that is possible. 